# Modelo lineal para la prediccion


In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer

In [45]:
df = pd.read_csv("../data/clean/personas_limpio.csv")

In [ ]:
def convert_to_categorical(df, columns):
    """
    Convierte las columnas especificadas de un DataFrame a tipo categórico.

    Parámetros:
    df (pd.DataFrame): El DataFrame en el que se realizarán las conversiones.
    columns (list): Lista de nombres de columnas a convertir a tipo categórico.

    Retorna:
    pd.DataFrame: El DataFrame con las columnas especificadas convertidas a tipo categórico.
    """
    for column in columns:
        if column in df.columns:
            df[column] = pd.Categorical(df[column]).codes
        else:
            print(f"Columna '{column}' no encontrada en el DataFrame.")
    return df

categorical_columns = ['gender', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'smoking_status','stroke']
one_hot_cols = ["gender","work_type","smoking_status"]
binary_cols = [x for x in categorical_columns if x not in one_hot_cols]
numeric_cols = [x for x in df.columns if x not in categorical_columns]

df = convert_to_categorical(df, binary_cols)

In [36]:
convert_to_categorical(df, categorical_columns).head(4)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,0,61.0,0,0,1,3,0,202.21,28.893237,2,1
2,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,0,49.0,0,0,1,2,1,171.23,34.400000,3,1


## Caso base

In [10]:
X = df.drop(columns=["stroke"])
y = df[["stroke"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
import mlflow
import mlflow.sklearn

In [56]:
categorical_columns = ['gender', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'smoking_status','stroke']
one_hot_cols = ["gender","work_type","smoking_status"]
binary_cols = [x for x in categorical_columns if x not in one_hot_cols]
numeric_cols = [x for x in X_train.columns if x not in categorical_columns]
pass_columns = [x for x in X_train.columns if x not in numeric_cols and x not in one_hot_cols]

print(pass_columns)

['hypertension', 'heart_disease', 'ever_married', 'Residence_type']


In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment = mlflow.set_experiment("project_linear_model")

with mlflow.start_run(run_name="modelo_base"):
    
    penalty = "l2"
    class_weight = None
    max_iter = 100


    preprocessor = ColumnTransformer(
        transformers=[
            ('scaler', StandardScaler(), numeric_cols),  
            ('onehot', OneHotEncoder(drop='first', handle_unknown="ignore"), one_hot_cols),
            ('passthrough', 'passthrough', pass_columns)
        ]
    )

    pipeline = Pipeline([
        ('prepros', preprocessor),  
        ('logreg', LogisticRegression(penalty=penalty, 
                                    class_weight=class_weight,
                                    max_iter=max_iter))  
    ])

    # Fitting the model on the training data
    pipeline.fit(X_train, y_train)

    # Predicting on the test data
    y_pred = pipeline.predict(X_test)


    # registrar parametros
    mlflow.log_param("penalty", penalty)
    mlflow.log_param("class_weight", class_weight)
    mlflow.log_param("max_iter", max_iter)

    # registrar el modelo
    mlflow.sklearn.log_model(pipeline, "logistic_regression_base")


    # Evaluating the model
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")

    f1_scr = f1_score(y_test, y_pred)
    print(f"f1_score: {f1_scr:.2f}")

    conf_mtrx = confusion_matrix(y_test,y_pred)
    print(f"confusion matrix: {conf_mtrx}")

    mlflow.log_metric("accuracy",accuracy)    
    mlflow.log_metric("f1_score",f1_scr)
    #mlflow.log_metric("confusion_matrix",conf_mtrx) 

c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
2024/11/09 15:31:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/09 15:31:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run modelo_base at: http://127.0.0.1:5000/#/experiments/634735473648121750/runs/919eebb2da8e4a04b8c342736f182991.
2024/11/09 15:31:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0

Accuracy: 0.94
f1_score: 0.00
confusion matrix: [[960   0]
 [ 62   0]]


In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment = mlflow.set_experiment("project_linear_model")

with mlflow.start_run(run_name="modelo_balanced"):

    penalty = "l2"
    class_weight = "balanced"
    max_iter = 100

    poly_columns = X_train.select_dtypes("float").columns.values
    not_poly_columns = [x for x in X_train.columns if x not in poly_columns]

    preprocessor = ColumnTransformer(
        transformers=[
            ('poly_scaled', Pipeline([
                ('poly', PolynomialFeatures(degree=2, include_bias=False)),  
                ('scaler', StandardScaler())]), poly_columns),  
            ('scaler', StandardScaler(), numeric_cols),  
            ('onehot', OneHotEncoder(drop='first', handle_unknown="ignore"), one_hot_cols),
            ('passthrough', 'passthrough', pass_columns)
        ]
    )

    pipeline = Pipeline([
        ('prepros', preprocessor),  
        ('logreg', LogisticRegression(penalty=penalty, 
                                    class_weight=class_weight,
                                    max_iter=max_iter))  
    ])

    # Fitting the model on the training data
    pipeline.fit(X_train, y_train)

    # Predicting on the test data
    y_pred = pipeline.predict(X_test)


    # registrar parametros
    mlflow.log_param("penalty", penalty)
    mlflow.log_param("class_weight", class_weight)
    mlflow.log_param("max_iter", max_iter)

    # registrar el modelo
    mlflow.sklearn.log_model(pipeline, "project_linear_model")


    # Evaluating the model
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")

    f1_scr = f1_score(y_test, y_pred)
    print(f"f1_score: {f1_scr:.2f}")

    conf_mtrx = confusion_matrix(y_test,y_pred)
    print(f"confusion matrix: {conf_mtrx}")

    mlflow.log_metric("accuracy",accuracy)    
    mlflow.log_metric("f1_score",f1_scr)
    #mlflow.log_metric("confusion_matrix",conf_mtrx)

c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
2024/11/09 15:31:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/09 15:31:11 INFO mlflow.tracking._tracking_service.client: 🏃 View run modelo_balanced at: http://127.0.0.1:5000/#/experiments/634735473648121750/runs/d728a7fefa884379907d01618e0e7750.
2024/11/09 15:31:11 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://1

Accuracy: 0.74
f1_score: 0.28
confusion matrix: [[705 255]
 [ 11  51]]


In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment = mlflow.set_experiment("project_linear_model")

with mlflow.start_run(run_name="modelo_var_polynomial_optimized"):
    
    penalty = None
    class_weight = "balanced"
    max_iter = 200
    poly_degree = 4


    poly_columns = X_train.select_dtypes("float").columns.values
    not_poly_columns = [x for x in X_train.columns if x not in poly_columns]

    preprocessor = ColumnTransformer(
        transformers=[
            ('poly_scaled', Pipeline([
                ('poly', PolynomialFeatures(degree=poly_degree, include_bias=False)),  
                ('scaler', StandardScaler())]), poly_columns),  
            ('scaler', StandardScaler(), numeric_cols),  
            ('onehot', OneHotEncoder(drop='first', handle_unknown="ignore"), one_hot_cols),
            ('passthrough', 'passthrough', pass_columns)
        ]
    )

    pipeline = Pipeline([
        ('prepros', preprocessor),  
        ('logreg', LogisticRegression(penalty=penalty, 
                                    class_weight=class_weight,
                                    max_iter=max_iter))  
    ])


    pipeline.fit(X_train, y_train)

    # Predicting on the test data
    y_pred = pipeline.predict(X_test)


    # registrar parametros
    mlflow.log_param("penalty", penalty)
    mlflow.log_param("class_weight", class_weight)
    mlflow.log_param("max_iter", max_iter)
    mlflow.log_param("poly_degree", poly_degree)

    # registrar el modelo
    mlflow.sklearn.log_model(pipeline, "project_linear_model")


    # Evaluating the model
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")

    f1_scr = f1_score(y_test, y_pred)
    print(f"f1_score: {f1_scr:.2f}")

    conf_mtrx = confusion_matrix(y_test,y_pred)
    print(f"confusion matrix: {conf_mtrx}")

    mlflow.log_metric("accuracy",accuracy)    
    mlflow.log_metric("f1_score",f1_scr)
    #mlflow.log_metric("confusion_matrix",conf_mtrx)

c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will 

Accuracy: 0.75
f1_score: 0.27
confusion matrix: [[713 247]
 [ 13  49]]


In [ ]:
poly_columns = X_train.select_dtypes("float").columns.values
not_poly_columns = [x for x in X_train.columns if x not in poly_columns]


penalty = None
class_weight = "balanced"
max_iter = 300
poly_degree = 4


param_grid = {"logreg__penalty": ["l2",None],
              "logreg__class_weight":[None, "balanced"],
              "logreg__max_iter": np.arange(100,400,50),
              "prepros__poly_scaled__poly__degree": [1, 2, 3, 4]}

poly_columns = X_train.select_dtypes("float").columns.values
not_poly_columns = [x for x in X_train.columns if x not in poly_columns]

preprocessor = ColumnTransformer(
    transformers=[
        ('poly_scaled', Pipeline([
            ('poly', PolynomialFeatures(degree=poly_degree, include_bias=False)),  
            ('scaler', StandardScaler())]), poly_columns),  
        ('scaler', StandardScaler(), numeric_cols),  
        ('onehot', OneHotEncoder(drop='first', handle_unknown="ignore"), one_hot_cols),
        ('passthrough', 'passthrough', pass_columns)
    ]
)

pipeline = Pipeline([
    ('prepros', preprocessor),  
    ('logreg', LogisticRegression(penalty=penalty, 
                                class_weight=class_weight,
                                max_iter=max_iter))  
])


pipeline.fit(X_train, y_train)

# Predicting on the test data
y_pred = pipeline.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

f1_scr = f1_score(y_test, y_pred)
print(f"f1_score: {f1_scr:.4f}")

confusion_matrix(y_test, y_pred)

Accuracy: 0.7397
f1_score: 0.2611


c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will 

array([[709, 251],
       [ 15,  47]], dtype=int64)

In [105]:
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=5, verbose=3)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=1;, score=0.051 total time=   0.0s
[CV 2/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=1;, score=0.000 total time=   0.0s
[CV 3/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=1;, score=0.000 total time=   0.0s
[CV 4/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=1;, score=0.000 total time=   0.0s
[CV 5/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=1;, score=0.000 total time=   0.0s
[CV 1/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=2;, score=0.000 total time=   0.0s
[CV 2/5] END logreg__class

c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expec

[CV 3/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=2;, score=0.000 total time=   0.0s
[CV 4/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=2;, score=0.000 total time=   0.0s
[CV 5/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=2;, score=0.000 total time=   0.0s
[CV 1/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=3;, score=0.000 total time=   0.0s
[CV 2/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=3;, score=0.000 total time=   0.0s
[CV 3/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=3;, score=0.000 total time=   0.0s
[CV 4/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepro

c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expec

[CV 1/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=4;, score=0.000 total time=   0.0s
[CV 2/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=4;, score=0.000 total time=   0.0s
[CV 3/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=4;, score=0.000 total time=   0.0s
[CV 4/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=4;, score=0.000 total time=   0.0s
[CV 5/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, prepros__poly_scaled__poly__degree=4;, score=0.000 total time=   0.0s
[CV 1/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=None, prepros__poly_scaled__poly__degree=1;, score=0.051 total time=   0.0s
[CV 2/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=None, pr

...

[CV 1/5] END logreg__class_weight=balanced, logreg__max_iter=350, logreg__penalty=None, prepros__poly_scaled__poly__degree=4;, score=0.201 total time=   0.0s
[CV 2/5] END logreg__class_weight=balanced, logreg__max_iter=350, logreg__penalty=None, prepros__poly_scaled__poly__degree=4;, score=0.231 total time=   0.0s
[CV 3/5] END logreg__class_weight=balanced, logreg__max_iter=350, logreg__penalty=None, prepros__poly_scaled__poly__degree=4;, score=0.212 total time=   0.0s


c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.


[CV 4/5] END logreg__class_weight=balanced, logreg__max_iter=350, logreg__penalty=None, prepros__poly_scaled__poly__degree=4;, score=0.232 total time=   0.0s
[CV 5/5] END logreg__class_weight=balanced, logreg__max_iter=350, logreg__penalty=None, prepros__poly_scaled__poly__degree=4;, score=0.254 total time=   0.0s


c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prepros',
                                        ColumnTransformer(transformers=[('poly_scaled',
                                                                         Pipeline(steps=[('poly',
                                                                                          PolynomialFeatures(degree=4,
                                                                                                             include_bias=False)),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         array(['age', 'avg_glucose_level', 'bmi'], dtype=object)),
                                                                        ('scaler',
                                                                         StandardScaler(),
                                                                         ['age',
                                                                          'avg_glucose_level',
                                                                          'bmi']),
                                                                        ('onehot',
                                                                         OneHotEncoder(drop...
                                                                         ['hypertension',
                                                                          'heart_disease',
                                                                          'ever_married',
                                                                          'Residence_type'])])),
                                       ('logreg',
                                        LogisticRegression(class_weight='balanced',
                                                           max_iter=300,
                                                           penalty=None))]),
             param_grid={'logreg__class_weight': [None, 'balanced'],
                         'logreg__max_iter': array([100, 150, 200, 250, 300, 350]),
                         'logreg__penalty': ['l2', None],
                         'prepros__poly_scaled__poly__degree': [1, 2, 3, 4]},
             scoring='f1', verbose=3)

In [110]:
grid_search.best_score_

0.22793021512379172

In [111]:
grid_search.best_params_

{'logreg__class_weight': 'balanced',
 'logreg__max_iter': 200,
 'logreg__penalty': None,
 'prepros__poly_scaled__poly__degree': 4}

In [112]:
# Evaluating the model

y_pred=grid_search.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

f1_scr = f1_score(y_test, y_pred)
print(f"f1_score: {f1_scr:.4f}")

Accuracy: 0.7456
f1_score: 0.2737


c:\Users\Paul\anaconda3\envs\tf-gpu2\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [113]:
confusion_matrix(y_test, y_pred)

array([[713, 247],
       [ 13,  49]], dtype=int64)